In [1]:
import os
import json
import hashlib
from dotenv import load_dotenv
from pinecone import Pinecone
from openai import OpenAI

In [2]:
load_dotenv()

True

In [3]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
pc =Pinecone(api_key=os.getenv("Pinecone_key"))


In [4]:
index = pc.Index("real-estate-ar-en")

In [7]:

def generate_embedding(text: str) -> list:
  
    response = client.embeddings.create(
        input=text,
        model="text-embedding-3-small"

    )
    return response.data[0].embedding

def upsert_to_pinecone(question: str, answer: str):

    combined_text = f"Q: {question}\nA: {answer}"

    # Create unique ID
    unique_id = hashlib.md5(combined_text.encode()).hexdigest()

    index.upsert(
        vectors=[{
            "id": unique_id,
            "values": generate_embedding(combined_text),
            "metadata": {
                "question": question,
                "answer": answer
            }
        }]
    )


with open('real_estate_knowledge_base_clean.json', 'r', encoding='utf-8') as f:
    qna_data = json.load(f)

for item in qna_data:
    upsert_to_pinecone(item["question"], item["answer"])

print("✅ Successfully loaded", len(qna_data), "Q&A pairs into Pinecone")

PermissionDeniedError: Error code: 403 - {'error': {'message': 'Project `proj_HebMPIXE6HciltEYnUjOnYKb` does not have access to model `text-embedding-3-small`', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [21]:
import openai
print(openai.Model.list())  

APIRemovedInV1: 

You tried to access openai.Model, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
